In [1]:
import nltk
import math
import random
import numpy as np
import pandas as pd
nltk.download('punkt')
with open("./en_US.twitter.txt","r") as f:
    data=f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
print(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
print(data[-300:])
print("-------")


[nltk_data] Downloading package punkt to /home/rishabh/nltk_data...


Data type: <class 'str'>
Number of letters: 138771
First 300 letters of the data
-------
How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
they've decided its more fun if I don't.
So Tired D; Played Lazer Tag & Ran A 
-------
Last 300 letters of the data
-------
..one of my most high traffic blog posts ever for sure
Woke up to 70 followers ( in 2 days) #blessed Thanks for ur support!
can't wait for your show in Dallas tonight! We'll be there with bells on, so to speak :)
Thanks for the follow Kevin!
Hm. When you don't know if an indirect tweets about you <<
-------


[nltk_data]   Package punkt is already up-to-date!


## Preprocessing

In [2]:
def split_to_sentences(data):
    sentences=data.split("\n")
    sentences=[s.strip() for s in sentences]
    sentences=[s for s in sentences if len(s)>0]
    return sentences

In [3]:
x = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""
print(x)

split_to_sentences(x)


I have a pen.
I have an apple. 
Ah
Apple pen.




['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']

In [4]:
def tokenize_sentences(sentences):
    tokenized_sentences=[]
    for sentence in sentences:
        sentence=sentence.lower()
        tokenized=nltk.word_tokenize(sentence)
        tokenized_sentences.append(tokenized)
    return tokenized_sentences

In [5]:
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

In [29]:
def get_tokenized_data(data):
    sentences=split_to_sentences(data)
    tokenized_sentences=tokenize_sentences(sentences)
    return tokenized_sentences

In [7]:
def count_words(tokenized_sentences):
    word_counts={}
    for sentence in tokenized_sentences:
        for token in sentence:
            if token not in word_counts:
                word_counts[token]=1
            else:
                word_counts[token]+=1
    return word_counts

In [8]:
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

In [10]:
def get_words_with_nplus_frequency(tokenized_sentences,count_threshold):
    closed_vocab=[]
    word_counts=count_words(tokenized_sentences)
    for word,cnt in word_counts.items():
        if cnt>=count_threshold:
            closed_vocab.append(word)
    return closed_vocab

In [13]:
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(tokenized_sentences, count_threshold=2)
print(f"Closed vocabulary:")
print(tmp_closed_vocab)


Closed vocabulary:
['.', 'are']


In [21]:
def replace_oov_words_by_unk(tokenized_sentences,vocabulary,unknown_token="<unk>"):
    vocabulary=set(vocabulary)
    replaced_tokenized_sentence=[]
    for sentence in tokenized_sentences:
        replaced_sentence=[]
        for token in sentence:
            if token in vocabulary:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentence.append(replaced_sentence)
    return replaced_tokenized_sentence

In [22]:
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(tokenized_sentences, vocabulary)
print(f"Original sentence:")
print(tokenized_sentences)
print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)

Original sentence:
[['dogs', 'run'], ['cats', 'sleep']]
tokenized_sentences with less frequent words converted to '<unk>':
[['dogs', '<unk>'], ['<unk>', 'sleep']]


In [23]:
def preprocess_data(train_data, test_data, count_threshold, unknown_token="<unk>",
                    get_words_with_nplus_frequency=get_words_with_nplus_frequency, replace_oov_words_by_unk=replace_oov_words_by_unk):
    vocabulary=get_words_with_nplus_frequency(train_data,count_threshold)
    train_data_replaced=replace_oov_words_by_unk(train_data,vocabulary,unknown_token)
    test_data_replaced=replace_oov_words_by_unk(test_data,vocabulary,unknown_token)
    return train_data_replaced,test_data_replaced,vocabulary

In [26]:
# test your code
tmp_train = [['sky', 'is', 'blue', '.'],
     ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train, 
                                                           tmp_test, 
                                                           count_threshold = 1
                                                          )

print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)


tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


In [30]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, 
                                                                        test_data, 
                                                                        minimum_freq)

In [31]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['it', "'s", '<unk>', 'again', '..', 'but', 'oh', 'well']

First preprocessed test sample:
['big', '<unk>', 'to', 'for', 'leaving', '<unk>', 'that', 'google', 'can', '<unk>', '<unk>', '.']

First 10 vocabulary:
['it', "'s", 'again', '..', 'but', 'oh', 'well', '``', 'them', 'the']

Size of vocabulary: 1702


## Develop_N_gram_Language_Model

In [32]:
def count_n_grams(data,n,start_token="<s>",end_token="<e>"):
    n_grams={}
    for sentence in data:
        sentence=[start_token]*n+sentence+[end_token]
        sentence=tuple(sentence)
        for i in range(len(sentence)-n+1):
            n_gram=sentence[i:i+n]
            if n_gram in n_grams.keys():
                n_grams[n_gram]+=1
            else:
                n_grams[n_gram]=1
    return n_grams

In [33]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


In [38]:
def estimate_probability(word,previous_n_gram,n_gram_counts,n_plus1_gram_counts,vocabulary_size,k=1.0):
    previous_n_gram=tuple(previous_n_gram)
    previous_n_gram_count=n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
    denominator=previous_n_gram_count+k*vocabulary_size
    n_plus1_gram=previous_n_gram+(word,)
    n_plus_1_gram_count=n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0
    numerator=n_plus_1_gram_count+k
    probability=numerator/denominator
    return probability

In [39]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
tmp_prob = estimate_probability("cat", ["a"], unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")


The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


In [40]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, end_token='<e>', unknown_token="<unk>",  k=1.0):
    previous_n_gram=tuple(previous_n_gram)
    vocabulary=vocabulary+[end_token,unknown_token]
    vocabulary_size=len(vocabulary)
    probabilities={}
    for word in vocabulary:
        probability=estimate_probability(word,previous_n_gram,n_gram_counts,n_plus1_gram_counts,vocabulary_size,k=k)
        probabilities[word]=probability
    return probabilities
        

In [41]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

estimate_probabilities(["a"], unigram_counts, bigram_counts, unique_words, k=1)

{'i': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'cat': 0.2727272727272727,
 'dog': 0.09090909090909091,
 'this': 0.09090909090909091,
 'a': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [42]:
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k=1)

{'i': 0.18181818181818182,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'cat': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'this': 0.18181818181818182,
 'a': 0.09090909090909091,
 '<e>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [46]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    # add <e> <unk> to the vocabulary
    # <s> is omitted since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    
    # obtain unique n-grams
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]        
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))
    
    # mapping from n-gram to row
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}    
    # mapping from next word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}    
    
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [47]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)

print('bigram counts')
display(make_count_matrix(bigram_counts, unique_words))

bigram counts


,i,is,like,cat,dog,this,a,<e>,<unk>
"(<s>,)",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(dog,)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a,)",0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
"(i,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(is,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(this,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(like,)",0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [48]:
print('\ntrigram counts')
trigram_counts = count_n_grams(sentences, 3)
display(make_count_matrix(trigram_counts, unique_words))



trigram counts


,i,is,like,cat,dog,this,a,<e>,<unk>
"(<s>, <s>)",1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(this, dog)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(a, cat)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(like, a)",0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
"(<s>, i)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>, this)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(i, like)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(is, like)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
"(dog, is)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix


In [50]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k=1))

bigram probabilities


,i,is,like,cat,dog,this,a,<e>,<unk>
"(<s>,)",0.181818,0.090909,0.090909,0.090909,0.090909,0.181818,0.090909,0.090909,0.090909
"(dog,)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(a,)",0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909
"(i,)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(is,)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(this,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(like,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909


## Perplexity

In [51]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, start_token='<s>', end_token = '<e>', k=1.0):
    n=len(list(n_gram_counts.keys())[0])
    sentence=[start_token]*n+sentence+[end_token]
    sentence=tuple(sentence)
    N=len(sentence)
    product_pi=1.0
    for t in range(n,N):
        n_gram=sentence[t-n:t]
        word=sentence[t]
        probability = estimate_probability(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)
        product_pi*=1/probability
    perplexity=(product_pi)**(1/N)
    return perplexity

In [52]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")

Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654


In [53]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, end_token='<e>', unknown_token="<unk>", k=1.0, start_with=None):
    n=len(list(n_gram_counts.keys())[0])
    previous_n_gram=previous_tokens[-n:]
    probabilities=estimate_probabilities(previous_n_gram,n_gram_counts,n_plus1_gram_counts,vocabulary,k=k)
    suggestion=None
    max_prob=0
    for word,prob in probabilities.items():
        if start_with:
            if not word.startswith(start_with):
                continue
        if prob>max_prob:
            suggestion=word
            max_prob=prob
    return suggestion,max_prob         

In [56]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")

The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909


In [57]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [58]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)


The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727),
 ('a', 0.2),
 ('i', 0.1111111111111111),
 ('i', 0.1111111111111111)]

In [66]:
# Assuming the necessary imports, n_gram_counts_list, and vocabulary have been defined earlier

# Function to get user input
def get_user_input():
    user_input = input("Enter a sequence of words: ")
    previous_tokens = user_input.split()  # Split the input string into individual tokens (words)
    return previous_tokens

# Function to get and display suggestions based on user input
def suggest_and_complete_sentence(n_gram_counts_list, vocabulary, k=1.0):
    previous_tokens = get_user_input()
    
    start_with = input("Optional: Enter a starting letter or sequence for the suggested word (or press Enter to skip): ")
    start_with = start_with if start_with != "" else None
    
    # Get word suggestions
    suggestions = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=k, start_with=start_with)
    
    # Find the suggestion with the highest probability
    if suggestions:
        best_suggestion, max_prob = max(suggestions, key=lambda x: x[1])
        if best_suggestion:
            # Append the best suggestion to the previous tokens
            completed_sentence = previous_tokens + [best_suggestion]
            print("\nCompleted Sentence:")
            print(" ".join(completed_sentence))
        else:
            print("No suitable suggestion found.")
    else:
        print("No suggestions available.")

# Call the function to suggest and complete the sentence based on user input
suggest_and_complete_sentence(n_gram_counts_list, vocabulary)


Enter a sequence of words:  I love
Optional: Enter a starting letter or sequence for the suggested word (or press Enter to skip):  y



Completed Sentence:
I love you
